In [1]:
# #Requeriments
# %pip install numpy==1.22
# %pip install pandas==1.5.0
# %pip install numpy scipy joblib scikit-learn --force-reinstall
%pip install sktime==0.14.0
# %pip install tensorflow
# %pip install tensorflow-estimator==2.5
# %pip install tensorflow-gpu==2.5

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import timeit
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import fbeta_score, make_scorer
import tensorflow as tf
from pickle import dump
from sklearn.model_selection import KFold


#Import Models
from sktime.classification.hybrid import HIVECOTEV2
from sktime.classification.hybrid import HIVECOTEV1
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import RandomIntervalSpectralEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.shapelet_based import ShapeletTransformClassifier
# from sktime._contrib.vector_classifiers._rotation_forest import RotationForest
from sktime.classification.kernel_based._rocket_classifier import RocketClassifier

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier


# from sktime.datasets import load_from_arff_to_dataframe as load_arff
#numpy.version.version

In [3]:
print(pd.__file__)
print(dir(pd))

D:\Programs\envs\RunModelsEnv\lib\site-packages\pandas\__init__.py
['ArrowDtype', 'BooleanDtype', 'Categorical', 'CategoricalDtype', 'CategoricalIndex', 'DataFrame', 'DateOffset', 'DatetimeIndex', 'DatetimeTZDtype', 'ExcelFile', 'ExcelWriter', 'Flags', 'Float32Dtype', 'Float64Dtype', 'Float64Index', 'Grouper', 'HDFStore', 'Index', 'IndexSlice', 'Int16Dtype', 'Int32Dtype', 'Int64Dtype', 'Int64Index', 'Int8Dtype', 'Interval', 'IntervalDtype', 'IntervalIndex', 'MultiIndex', 'NA', 'NaT', 'NamedAgg', 'Period', 'PeriodDtype', 'PeriodIndex', 'RangeIndex', 'Series', 'SparseDtype', 'StringDtype', 'Timedelta', 'TimedeltaIndex', 'Timestamp', 'UInt16Dtype', 'UInt32Dtype', 'UInt64Dtype', 'UInt64Index', 'UInt8Dtype', '__all__', '__builtins__', '__cached__', '__deprecated_num_index_names', '__dir__', '__doc__', '__docformat__', '__file__', '__getattr__', '__git_version__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_config', '_is_numpy_dev', '_libs', '_testing', 

In [4]:
if tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow includes GPU support.")
else:
    print("The installed version of TensorFlow does not include GPU support.")

The installed version of TensorFlow does not include GPU support.


In [5]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [6]:
tf.config.list_physical_devices('GPU')

[]

In [7]:
# %pip install tsai

In [8]:
#%pip install sklearn --upgrade

In [9]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [10]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   print(
#       '\n\nThis error most likely means that this notebook is not '
#       'configured to use a GPU.  Change this in Notebook Settings via the '
#       'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
#   raise SystemError('GPU device not found')

In [11]:
def create_dt_metrics():
    columns = ['TimeSeriesForestClassifier','HIVECOTE1','HIVECOTE2','RISE','BOSSEnsemble','ROCKET'
               ,'KNeighborsTimeSeriesClassifier','ShapeletTransformClassifier']
    df_precision = pd.DataFrame(data= None,
                           columns=columns)
    df_accuracy  = pd.DataFrame(data= None,
                           columns=columns)
    df_f1        = pd.DataFrame(data= None,
                           columns=columns)
    df_recall    = pd.DataFrame(data= None,
                           columns=columns)
    df_std       = pd.DataFrame(data= None,
                           columns=columns)
    return df_precision, df_accuracy, df_f1, df_recall, df_std

In [12]:
def populate_df_metrics(precision, accuracy, f1, recall):
    print(f"Precision in populate_df_metrics: {precision}")
    df_precision = df_precision.append(precision, ignore_index=True)
    print(f"Df precision after append: {df_precision}")
    df_accuracy.append(pd.DataFrame(accuracy))
    df_f1.append(pd.DataFrame(f1))
    df_recall.append(pd.DataFrame(recall))
  
def save_std(df_accuracy,df_std):
    for row in df_accuracy:
        df_std[row] = np.std(df_accuracy[row])
        print(f"STD COL: {df_std[row]}")
    print(f"STD antes do %save: {df_std}")
    df_std.to_csv("results4/" + 'df_all_std.csv', index=False)
    print(f"Save STD: {df_std}")

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
def create_models(models):
#       n_estimators=200, n_estimators=6, n_estimators=400   
    models.append(('TimeSeriesForestClassifier',TimeSeriesForestClassifier()))
# #   n_estimators=3, n_estimators=200, n_estimators=400    
    models.append(('HIVECOTE1', HIVECOTEV1(random_state=2)))
    
#   n_estimators=3, n_estimators=200, n_estimators=400    
    models.append(('HIVECOTE2',HIVECOTEV2(time_limit_in_minutes=0.2)))
#   n_estimators=50, n_estimators=3, n_estimators=100 
    models.append(('RISE',RandomIntervalSpectralEnsemble()))
#   max_ensemble_size=50, max_ensemble_size=3, max_ensemble_size=100
#   DICA: Geralmente, o max_win_len_prop é o dobro do min_window
    models.append(('BOSSEnsemble',BOSSEnsemble(max_ensemble_size=3, min_window =9, max_win_len_prop=18)))
#   num_kernels=10000, num_kernels=5000, num_kernels=20000 
    models.append(('ROCKET', RocketClassifier()))
    #     #   n_neighbors=1, n_neighbors=5, n_neighbors=10
    models.append(('KNeighborsTimeSeriesClassifier',KNeighborsTimeSeriesClassifier()))
    #     n_estimators=200, n_estimators=6, n_estimators=400     
    models.append(('ShapeletTransformClassifier',ShapeletTransformClassifier(
        n_shapelet_samples=500,
        max_shapelets=20,
        batch_size=100,
    )))
    return models

In [14]:
# model = make_pipeline(StandardScaler(with_mean=False),RocketClassifier())
# model.get_params().keys()

In [15]:
#save metrics
def save_results(df_to_save, name, model, train_val_test):
    #save the model
    print("Saving model...")
    dump(model, open('results5/'+name+'_'+train_val_test+'_model2.pkl', 'wb'))
    print("Saving metrics...")
    print(f"Df accuracy: {df_to_save}")
    df_to_save.to_csv("results5/" + 'df_'+name+'_'+train_val_test+'2.csv', index=False)
    print("Accuracy of models:")
    print(df_to_save)

In [16]:
#%pip install -U setuptools pip

In [17]:
#%pip install cuda-python

In [18]:
#%pip install cupy-cuda11x -f https://pip.cupy.dev/aarch64

In [19]:
#%pip install -c numba -c conda-forge -c defaults cudf

In [20]:
# import cupy as cp

In [21]:
# from numba import jit, cuda, vectorize, float32

In [22]:
# print(cuda.gpus)

In [23]:
rootPath = r"C:\Users\mlb\TSC_Data\newDataSet"
testeDataFrame = "\AllCandida_TEST.csv"
trainDataFrame = "\AllCandida_TRAIN.csv"
valDataFrame = "\AllCandida_VAL.csv"

In [24]:
from numpy.random import default_rng

from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested,
    from_nested_to_2d_array,
)
from sktime.datatypes._panel._check import is_nested_dataframe

In [25]:
candida_data_train  = pd.read_csv(rootPath+trainDataFrame, sep=",")
# 652 is the column of label
y_train = candida_data_train['652']
X_train_1 = candida_data_train.drop(columns=['652'])

candida_data_test  = pd.read_csv(rootPath+testeDataFrame, sep=",")
y_test = candida_data_test['652']
X_test_1 = candida_data_test.drop(columns=['652'])

bacteria_data_val  = pd.read_csv(rootPath+valDataFrame, sep=",")
y_val = bacteria_data_val['652']
X_val_1 = bacteria_data_val.drop(columns=['652'])

print(f"Treino {len(X_train_1)}")
print(f"Teste {len(X_test_1)}")
print(f"X_val_1 {len(X_val_1)}")

Treino 238
Teste 80
X_val_1 79


In [27]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_s = pd.DataFrame(scaler.fit_transform(X_train_1))
X_train_s

,0,1,2,3,4,5,6,7,8,9,...,642,643,644,645,646,647,648,649,650,651
0,0.097089,0.888026,0.409823,0.211322,0.011470,0.002984,0.094389,0.097508,0.888311,0.412353,...,0.006147,0.211099,0.165937,0.755979,0.942633,0.147668,0.017130,0.006142,0.211215,0.6
1,0.834456,0.001447,0.038346,0.578831,0.709699,0.613095,0.976286,0.836283,0.001452,0.037597,...,0.299426,0.882699,0.605010,0.087055,0.354621,0.506213,0.470051,0.301036,0.879555,0.6
2,0.846607,0.001447,0.001259,0.637470,0.852780,0.833073,0.995040,0.842061,0.001452,0.001000,...,0.542005,0.998041,0.671954,0.088129,0.046484,0.561678,0.766249,0.546414,0.997163,1.0
3,0.032964,0.824803,0.370739,0.066015,0.018003,0.005947,0.090825,0.033225,0.825896,0.373669,...,0.015825,0.224374,0.069241,0.649428,0.853630,0.041059,0.037784,0.016002,0.224307,0.8
4,0.137392,0.833933,0.523603,0.256131,0.024064,0.007370,0.158710,0.138076,0.834368,0.526231,...,0.017293,0.304897,0.203347,0.689033,0.955287,0.184945,0.041651,0.017419,0.305259,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,0.053176,0.819635,0.450645,0.101789,0.017146,0.006038,0.123063,0.053329,0.820363,0.453210,...,0.013421,0.266376,0.093962,0.658882,0.940946,0.085359,0.033602,0.013576,0.266856,0.8
234,0.724615,0.151579,0.552265,0.878849,0.306767,0.195811,0.787973,0.731672,0.149882,0.547964,...,0.463643,0.984766,0.853424,0.089202,0.105496,0.873582,0.580337,0.467981,0.985817,0.6
235,0.170837,0.682854,0.685152,0.276231,0.052296,0.022295,0.293707,0.172024,0.683777,0.688790,...,0.049636,0.505114,0.248195,0.493305,0.915215,0.206555,0.097539,0.050178,0.506437,0.6
236,0.247714,0.663216,0.725540,0.340955,0.061548,0.027736,0.336175,0.248706,0.663376,0.728344,...,0.050355,0.505332,0.288121,0.530100,0.909309,0.253377,0.090435,0.051028,0.507310,0.8


In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_test_s = pd.DataFrame(scaler.fit_transform(X_test_1))
X_test_s

,0,1,2,3,4,5,6,7,8,9,...,642,643,644,645,646,647,648,649,650,651
0,0.029108,0.813493,0.475749,0.038986,0.019014,0.006149,0.086889,0.029660,0.813884,0.478888,...,0.018380,0.136913,0.007596,0.776878,0.857653,0.029664,0.039080,0.018702,0.137193,0.8
1,0.034635,0.818138,0.496219,0.055261,0.019581,0.006241,0.093561,0.035298,0.818719,0.498778,...,0.018099,0.136430,0.008070,0.801356,0.859352,0.042351,0.038282,0.018220,0.136463,0.8
2,0.155122,0.697011,0.861830,0.226911,0.050652,0.020759,0.269356,0.155534,0.697346,0.862477,...,0.055422,0.422351,0.180275,0.621226,0.930314,0.185821,0.096918,0.055665,0.419849,0.6
3,0.073446,0.839473,0.677034,0.252271,0.021784,0.008316,0.157952,0.073906,0.840300,0.680059,...,0.023076,0.270924,0.084026,0.824578,0.992351,0.190485,0.044535,0.023046,0.269764,0.8
4,0.834316,0.000120,0.359754,0.445307,0.628219,0.478822,0.962452,0.840422,0.000000,0.358982,...,0.237660,0.769956,0.511156,0.159386,0.537690,0.368470,0.354623,0.237300,0.768426,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.915623,0.000000,0.000967,0.461014,0.962853,0.997525,0.998604,0.905503,0.000000,0.000625,...,0.571394,0.965409,0.647164,0.000000,0.123608,0.376493,0.683851,0.570044,0.965945,0.8
76,0.026406,0.820891,0.464377,0.042581,0.018101,0.005915,0.082545,0.026719,0.820963,0.466955,...,0.016775,0.114175,0.001305,0.789745,0.842356,0.022761,0.037325,0.016973,0.115544,0.8
77,0.856792,0.000964,0.003923,0.801665,0.861172,0.759212,0.936850,0.856600,0.000967,0.003921,...,0.782487,0.977987,0.805364,0.109553,0.012110,0.875187,0.882601,0.783212,0.979324,0.6
78,0.858757,0.000964,0.001990,0.648372,0.855821,0.854678,0.980760,0.851452,0.000967,0.001989,...,0.735934,0.989840,0.678614,0.108674,0.026260,0.600746,0.853251,0.736556,0.993189,1.0


In [29]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_val_s = pd.DataFrame(scaler.fit_transform(X_val_1))
X_val_s

,0,1,2,3,4,5,6,7,8,9,...,642,643,644,645,646,647,648,649,650,651
0,0.932939,0.000754,0.002617,0.852785,0.801356,0.709076,0.955173,0.933627,0.000756,0.002272,...,0.706696,0.982792,0.817108,0.112773,0.002920,0.911232,0.800911,0.707207,0.984645,0.6
1,0.170735,0.727291,0.887353,0.294824,0.046314,0.018252,0.281681,0.171411,0.727356,0.888681,...,0.041733,0.389481,0.167959,0.682776,0.925932,0.202018,0.077150,0.041814,0.389471,0.6
2,0.044540,0.869925,0.531774,0.117103,0.018266,0.006299,0.108888,0.044710,0.869527,0.533140,...,0.019076,0.177896,0.039472,0.845587,0.874296,0.091759,0.038768,0.019258,0.177187,0.8
3,0.033342,0.864363,0.486829,0.072623,0.016641,0.005302,0.090740,0.033249,0.865028,0.489976,...,0.015105,0.132816,0.019910,0.834281,0.856943,0.046160,0.032362,0.015309,0.133561,0.8
4,0.003649,0.930387,0.263242,0.030309,0.000000,0.000000,0.000000,0.003275,0.930174,0.265633,...,0.006673,0.112458,0.074083,0.828466,0.916198,0.022986,0.008753,0.007377,0.114550,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.780070,0.144554,0.721227,0.967526,0.288902,0.177449,0.797115,0.788413,0.143556,0.714318,...,0.513331,0.987882,0.879616,0.112773,0.080882,0.949355,0.583874,0.515276,0.989764,0.6
75,0.235908,0.620898,0.981225,0.273371,0.061054,0.028380,0.365286,0.237154,0.620818,0.981257,...,0.097659,0.625788,0.323533,0.435328,0.791764,0.241450,0.146265,0.097891,0.626615,0.6
76,0.969552,0.000754,0.001309,0.984452,0.764437,0.737026,0.946177,0.974937,0.000756,0.001306,...,0.786451,0.981823,0.836787,0.112773,0.000973,0.982059,0.796493,0.787505,0.983670,0.6
77,0.866004,0.000251,0.159754,0.494588,0.580119,0.444900,0.967272,0.867128,0.000252,0.158062,...,0.222598,0.799806,0.527376,0.129926,0.483218,0.397122,0.345575,0.222320,0.797709,0.6


In [30]:
X_test = from_2d_array_to_nested(X_test_s)
print(f"X_nested is a nested DataFrame: {is_nested_dataframe(X_test)}")
print(f"The cell contains a {type(X_test.iloc[0,0])}.")
print(f"The nested DataFrame has shape {X_test.shape}")
X_test.head()

X_nested is a nested DataFrame: True
The cell contains a <class 'pandas.core.series.Series'>.
The nested DataFrame has shape (80, 1)


,0
0,0 0.029108 1 0.813493 2 0.47574...
1,0 0.034635 1 0.818138 2 0.49621...
2,0 0.155122 1 0.697011 2 0.86183...
3,0 0.073446 1 0.839473 2 0.67703...
4,0 0.834316 1 0.000120 2 0.35975...


In [31]:
X_train = from_2d_array_to_nested(X_train_s)
print(f"X_nested is a nested DataFrame: {is_nested_dataframe(X_train)}")
print(f"The cell contains a {type(X_train.iloc[0,0])}.")
print(f"The nested DataFrame has shape {X_train.shape}")
X_train.head()

X_nested is a nested DataFrame: True
The cell contains a <class 'pandas.core.series.Series'>.
The nested DataFrame has shape (238, 1)


,0
0,0 0.097089 1 0.888026 2 0.40982...
1,0 0.834456 1 0.001447 2 0.03834...
2,0 0.846607 1 0.001447 2 0.00125...
3,0 0.032964 1 0.824803 2 0.37073...
4,0 0.137392 1 0.833933 2 0.52360...


In [32]:
X_val = from_2d_array_to_nested(X_val_s)
print(f"X_nested is a nested DataFrame: {is_nested_dataframe(X_val)}")
print(f"The cell contains a {type(X_val.iloc[0,0])}.")
print(f"The nested DataFrame has shape {X_val.shape}")
X_val.head()

X_nested is a nested DataFrame: True
The cell contains a <class 'pandas.core.series.Series'>.
The nested DataFrame has shape (79, 1)


,0
0,0 0.932939 1 0.000754 2 0.00261...
1,0 0.170735 1 0.727291 2 0.88735...
2,0 0.044540 1 0.869925 2 0.53177...
3,0 0.033342 1 0.864363 2 0.48682...
4,0 0.003649 1 0.930387 2 0.26324...


In [39]:
import copy
# import cupy as cp
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
print(type(X_train))
print(len(X_test))

models =  []
models = create_models(models)

def gpu(kf, models):
    scoring = ['accuracy','precision_macro', 'recall_macro','f1_macro']
    df_precision, df_accuracy, df_f1, df_recall, df_std = create_dt_metrics()
#     with tf.device('/device:GPU:0'):
    df_precision, df_accuracy, df_f1, df_recall, df_std = create_dt_metrics()
    print(f"Quantidade de modelos {len(models)}")
    for name, model in models:
        print(f"Treining {name}...")
        print("In trainning***")
#             with cp.cuda.Device(0):
#                 cuda.to_device(X_train)
#                 cuda.to_device(y_train)
        model.fit(X_train, y_train)
        print("In prediction...")
        df_train = cross_validate(model, X_test, y_test, cv=kf, n_jobs=-1,scoring=scoring
                                          ,return_train_score=True)
        print("Finished prediction!")
        print(df_train.keys())
        save_results(pd.DataFrame(df_train),name, model,'trainAndVal')
        print("***********")
        print()
#             df_validation = cross_validate(model, X_val, y_val, cv=kf, scoring=scoring)
#             print(df_validation.keys())
#             save_results(pd.DataFrame(df_validation), name, model,'test2')


<class 'pandas.core.frame.DataFrame'>
80


In [ ]:
if __name__=="__main__":
    if tf.config.list_physical_devices('GPU'):
        print("Using MirroredStrategy")
        strategy = tf.distribute.MirroredStrategy()
    else:  # Use the Default Strategy
        print("Using Default Strategy")
        strategy = tf.distribute.get_strategy()
    with strategy.scope():
        # Do something interesting
        print(tf.Variable(1.))
        gpu(kf,models)

### Executar só no final

In [ ]:
def test_final_model(X_test, y_test, model):
    scoring = ['accuracy','precision_macro', 'recall_macro','f1_macro']
#     for name, model in models:
        # df_test = cross_validate(model, X_test, y_test, cv=kf, scoring=scoring
        #                          , return_train_score=True)
#         print(f'Model in test: {name}')
    df_test = pd.DataFrame([], columns=['accuracy','recall','precision','f1_score'])
    y_pred = model.predict(X_test)
    accuracy  =  accuracy_score(y_test, y_pred)
    recall    =  recall_score(y_test, y_pred, average='macro')
    precision =  precision_score(y_test, y_pred, average='macro')
    f1  =  f1_score(y_test, y_pred, average='macro')
    print(f"accuracy: {accuracy}, recall: {recall}, precision: {precision},f1: {f1} ")
    df_test.at[0, 'accuracy'] = accuracy
    df_test.at[0, 'recall'] = recall
    df_test.at[0,'precision'] = precision
    df_test.at[0,'f1_score'] = f1
    print(df_test.keys())
    save_results(df_test,"HIVECOTE2", model,'test')
    print("Finished test!")

In [ ]:
# X_test, y_test = load_arff("AllCandidas_TEST_V3.arff")
print(len(X_val))
# for name, model in models:
test_final_model(X_val,y_val,loaded_model) #Executar isso só no final

In [ ]:
def test_final_model(X_test, y_test, models):
    scoring = ['accuracy','precision_macro', 'recall_macro','f1_macro']
    for name, model in models:
        df_test = cross_validate(model, X_test, y_test, cv=kf, scoring=scoring
                                 , return_train_score=True)
        print(df_test.keys())
        save_results(pd.DataFrame(df_test),name, model,'test')
        print("Finished test!")